In [18]:
from hvec_importers.rws import communicators as com
from hvec_importers.rws import rws
from hvec_importers.rws import helpers as hlp
from hvec_importers.rws import parsers as parse
import dateutil
import pandas as pd
import requests
import warnings
import swifter

In [19]:
com.TIMEOUT

600

Understanding the raw data structure of data imported from RWS Waterinfo. Goal is to optimise the parsing procedure. The one in ddlpy relies on nested for-loops and is therefore very slow.

Get some data first. A deep dive in the library is necessary to obtain some raw data to experiment on. Do not try this at home!

In [20]:
warnings.simplefilter('ignore')

In [21]:
locations = rws.station_list()

In [42]:
sel = hlp.create_selection_table(
    locations = locations,
    name = 'Vlissingen', quantity = 'WATHTE')

In [43]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
0,VLIS,19305,25831,541518.745920,5.699255e+06,Vlissingen,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,1800-01-01,2100-12-31
1,VLISSGN,20536,25831,541425.983215,5.699182e+06,Vlissingen,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,1800-01-01,2100-12-31


In [44]:
code = 'VLISSGN'
sel = sel.query('Code == @code')

In [45]:
start = dateutil.parser.parse('1700-01-01')
end = dateutil.parser.parse('2022-1-3')

In [46]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
1,VLISSGN,20536,25831,541425.983215,5.699182e+06,Vlissingen,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,1800-01-01,2100-12-31


In [47]:
CHUNK = 8

In [48]:
starts = pd.date_range(start, end, freq = f'{CHUNK}MS').to_pydatetime()

In [49]:
starts

array([datetime.datetime(1678, 1, 1, 0, 0),
       datetime.datetime(1678, 9, 1, 0, 0),
       datetime.datetime(1679, 5, 1, 0, 0),
       datetime.datetime(1680, 1, 1, 0, 0),
       datetime.datetime(1680, 9, 1, 0, 0),
       datetime.datetime(1681, 5, 1, 0, 0),
       datetime.datetime(1682, 1, 1, 0, 0),
       datetime.datetime(1682, 9, 1, 0, 0),
       datetime.datetime(1683, 5, 1, 0, 0),
       datetime.datetime(1684, 1, 1, 0, 0),
       datetime.datetime(1684, 9, 1, 0, 0),
       datetime.datetime(1685, 5, 1, 0, 0),
       datetime.datetime(1686, 1, 1, 0, 0),
       datetime.datetime(1686, 9, 1, 0, 0),
       datetime.datetime(1687, 5, 1, 0, 0),
       datetime.datetime(1688, 1, 1, 0, 0),
       datetime.datetime(1688, 9, 1, 0, 0),
       datetime.datetime(1689, 5, 1, 0, 0),
       datetime.datetime(1690, 1, 1, 0, 0),
       datetime.datetime(1690, 9, 1, 0, 0),
       datetime.datetime(1691, 5, 1, 0, 0),
       datetime.datetime(1692, 1, 1, 0, 0),
       datetime.datetime(1692, 9

In [50]:
ends = starts + dateutil.relativedelta.relativedelta(months = CHUNK, days = -1)

In [51]:
ends

array([datetime.datetime(1678, 8, 31, 0, 0),
       datetime.datetime(1679, 4, 30, 0, 0),
       datetime.datetime(1679, 12, 31, 0, 0),
       datetime.datetime(1680, 8, 31, 0, 0),
       datetime.datetime(1681, 4, 30, 0, 0),
       datetime.datetime(1681, 12, 31, 0, 0),
       datetime.datetime(1682, 8, 31, 0, 0),
       datetime.datetime(1683, 4, 30, 0, 0),
       datetime.datetime(1683, 12, 31, 0, 0),
       datetime.datetime(1684, 8, 31, 0, 0),
       datetime.datetime(1685, 4, 30, 0, 0),
       datetime.datetime(1685, 12, 31, 0, 0),
       datetime.datetime(1686, 8, 31, 0, 0),
       datetime.datetime(1687, 4, 30, 0, 0),
       datetime.datetime(1687, 12, 31, 0, 0),
       datetime.datetime(1688, 8, 31, 0, 0),
       datetime.datetime(1689, 4, 30, 0, 0),
       datetime.datetime(1689, 12, 31, 0, 0),
       datetime.datetime(1690, 8, 31, 0, 0),
       datetime.datetime(1691, 4, 30, 0, 0),
       datetime.datetime(1691, 12, 31, 0, 0),
       datetime.datetime(1692, 8, 31, 0, 0),
   

In [52]:
def range_from_boolean():
    session = requests.session()
    range = []
    for (strt, nd) in list(zip(starts, ends)):
        data_present = com.assert_data_available(sel, strt, nd, session)
        if data_present:
            range.append(list(zip([strt], [nd])))
    return range

In [53]:
range_from_boolean()

[[(datetime.datetime(1812, 1, 1, 0, 0), datetime.datetime(1812, 8, 31, 0, 0))],
 [(datetime.datetime(1812, 9, 1, 0, 0), datetime.datetime(1813, 4, 30, 0, 0))],
 [(datetime.datetime(1813, 5, 1, 0, 0),
   datetime.datetime(1813, 12, 31, 0, 0))],
 [(datetime.datetime(1814, 1, 1, 0, 0), datetime.datetime(1814, 8, 31, 0, 0))],
 [(datetime.datetime(1814, 9, 1, 0, 0), datetime.datetime(1815, 4, 30, 0, 0))],
 [(datetime.datetime(1815, 5, 1, 0, 0),
   datetime.datetime(1815, 12, 31, 0, 0))],
 [(datetime.datetime(1816, 1, 1, 0, 0), datetime.datetime(1816, 8, 31, 0, 0))],
 [(datetime.datetime(1816, 9, 1, 0, 0), datetime.datetime(1817, 4, 30, 0, 0))],
 [(datetime.datetime(1817, 5, 1, 0, 0),
   datetime.datetime(1817, 12, 31, 0, 0))],
 [(datetime.datetime(1862, 1, 1, 0, 0), datetime.datetime(1862, 8, 31, 0, 0))],
 [(datetime.datetime(1862, 9, 1, 0, 0), datetime.datetime(1863, 4, 30, 0, 0))],
 [(datetime.datetime(1863, 5, 1, 0, 0),
   datetime.datetime(1863, 12, 31, 0, 0))],
 [(datetime.datetime(186